In [1]:
import sys

from torch import nn
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v3_large
from torchvision.transforms import Resize
import torch

sys.path.insert(0, '..')

## View MobileNetV3 Architecture for Reference

In [2]:
# weights = MobileNet_V3_Large_Weights.IMAGENET1K_V2
model = mobilenet_v3_large(pretrained=True)

print(model)

MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False

## Remove Last Layer

In [3]:
model_submodule_excl_last = list(model.children())[:-1]
model_submodule_excl_last = nn.Sequential(*model_submodule_excl_last)

model_submodule_excl_last

Sequential(
  (0): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [4]:
model_submodule_excl_last(torch.rand(10, 3, 224, 224)).shape

torch.Size([10, 960, 1, 1])

In [5]:
model_last_submodule = list(list(model.children())[-1].children())[:-1]  # Remove last layer from last submodule
model_last_submodule = nn.Sequential(*model_last_submodule)

model_last_submodule

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
)

In [6]:
# output = model_submodule_excl_last(torch.rand(10, 3, 224, 224))
# model_last_submodule(output)

In [7]:
output = model_submodule_excl_last(torch.rand(10, 3, 224, 224))
output = torch.flatten(output, 1)
model_last_submodule(output).shape

torch.Size([10, 1280])

## Import `SiameseDuplicateImageNetwork` for Testing

In [8]:
from scripts.dataset_duplicate_image import DuplicateImageDataset
from scripts.model import SiameseDuplicateImageNetwork

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [10]:
model = SiameseDuplicateImageNetwork().to(device)

In [11]:
print(model)

SiameseDuplicateImageNetwork(
  (model): Sequential(
    (0): SiameseNetwork(
      (net1): Sequential(
        (0): Sequential(
          (0): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): Hardswish()
            )
            (1): InvertedResidual(
              (block): Sequential(
                (0): ConvNormActivation(
                  (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
                  (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                  (2): ReLU(inplace=True)
                )
                (1): ConvNormActivation(
                  (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (1): BatchNorm2d(16, eps=0.001, mom

In [12]:
train_dataset = DuplicateImageDataset('../data/Training Data', transforms=[Resize((224, 224), antialias=True)])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [13]:
feature1, feature2, _ = next(iter(train_dataloader))
feature1, feature2

(tensor([[[[159, 159, 158,  ..., 185, 184, 183],
           [159, 159, 158,  ..., 185, 184, 183],
           [159, 157, 157,  ..., 185, 183, 181],
           ...,
           [125, 125, 127,  ..., 115, 111, 111],
           [125, 124, 126,  ..., 110, 113, 112],
           [125, 124, 126,  ...,  71, 112, 114]],
 
          [[179, 179, 178,  ..., 177, 176, 175],
           [179, 179, 178,  ..., 177, 176, 175],
           [179, 177, 177,  ..., 177, 175, 173],
           ...,
           [142, 142, 141,  ...,  90,  87,  86],
           [142, 141, 141,  ...,  88,  88,  87],
           [142, 141, 141,  ...,  53,  86,  88]],
 
          [[203, 203, 202,  ..., 166, 165, 164],
           [203, 203, 202,  ..., 166, 165, 164],
           [203, 201, 201,  ..., 166, 164, 162],
           ...,
           [152, 152, 152,  ...,  67,  64,  64],
           [152, 151, 152,  ...,  68,  68,  67],
           [152, 151, 152,  ...,  42,  68,  71]]],
 
 
         [[[196, 195, 196,  ..., 158, 157, 157],
         

In [14]:
feature1 = feature1.to(device)
feature2 = feature2.to(device)

output = model(feature1, feature2)
print(output)

tensor([[-0.0772],
        [-0.0772],
        [-0.0780],
        [-0.0761],
        [-0.0753],
        [-0.0746],
        [-0.0768],
        [-0.0743],
        [-0.0762],
        [-0.0750],
        [-0.0762],
        [-0.0747],
        [-0.0757],
        [-0.0756],
        [-0.0758],
        [-0.0769],
        [-0.0785],
        [-0.0741],
        [-0.0755],
        [-0.0754],
        [-0.0756],
        [-0.0759],
        [-0.0780],
        [-0.0768],
        [-0.0803],
        [-0.0776],
        [-0.0752],
        [-0.0752],
        [-0.0750],
        [-0.0779],
        [-0.0796],
        [-0.0747],
        [-0.0763],
        [-0.0779],
        [-0.0783],
        [-0.0762],
        [-0.0793],
        [-0.0753],
        [-0.0772],
        [-0.0768],
        [-0.0756],
        [-0.0746],
        [-0.0775],
        [-0.0766],
        [-0.0766],
        [-0.0766],
        [-0.0753],
        [-0.0770],
        [-0.0785],
        [-0.0773],
        [-0.0749],
        [-0.0752],
        [-0.

In [15]:
output.shape

torch.Size([64, 1])